In [2]:
import numpy as np
import pandas as pd

import math

import warnings
warnings.filterwarnings('ignore')

##Чтение данных

In [1]:
from google.colab import files
files.upload()

Saving data.txt to data.txt


{'data.txt': b'2x1 + 3x2 -> max\nx1 + 5x2 \xe2\x89\xa4 290\n2x1 + 4x2 \xe2\x89\xa4 430\n4x1 + x2 \xe2\x89\xa4 360\nx1 \xe2\x89\xa5 0\nx2 \xe2\x89\xa5 0\n'}

In [16]:
opt_dir = 0
f, A, b, b_signs = [], [], [], []

with open('data.txt', 'r') as fd:
  obj_func = fd.readline()
  if obj_func[-4:] == 'max\n':
    opt_dir = 1
  f_x1 = obj_func[:obj_func.find('x1')]
  if f_x1 == "": f_x1 = 1
  tmp = obj_func[:obj_func.find('x2')]
  f_x2 = tmp[obj_func.find('+')+2:]
  if f_x2 == "": f_x2 = 1
  f.append(int(f_x1))
  f.append(int(f_x2))

  print(obj_func, end='')

  bnd = fd.readline()
  while bnd:
    print(bnd, end='')

    sign = bnd.find('≤')

    x1_idx = bnd.find('x1')
    x2_idx = bnd.find('x2')
    if (x1_idx != -1) and (x2_idx != -1):
      coef_x1 = bnd[:x1_idx]
      if coef_x1 == "":
        coef_x1 = 1

      coef_x2 = bnd[:x2_idx]
      x2_idx = coef_x2.find('+')
      coef_x2 = coef_x2[x2_idx+2:]
      if coef_x2 == "":
        coef_x2 = 1

      if sign != -1:
        A.append([int(coef_x1), int(coef_x2)])
        b_val = bnd[sign+2:len(bnd)-1]
        b.append(int(b_val))
        b_signs.append('lt')
      else:
        A.append([-int(coef_x1), -int(coef_x2)])
        b.append(-int(bnd[bnd.find('≥')+2:len(bnd)-1]))
        b_signs.append('gt')

    bnd = fd.readline()

2x1 + 3x2 -> max
x1 + 5x2 ≤ 290
2x1 + 4x2 ≤ 430
4x1 + x2 ≤ 360
x1 ≥ 0
x2 ≥ 0


In [17]:
A = np.array(A)
new_A = np.hstack((A, np.eye(len(b_signs))))
for n, s in enumerate(b_signs):
  f.append(0)
  if s == 'gt':
    new_A[n, n] = -1
A = new_A.astype(int).tolist()

In [18]:
f

[2, 3, 0, 0, 0]

In [19]:
A

[[1, 5, 1, 0, 0], [2, 4, 0, 1, 0], [4, 1, 0, 0, 1]]

In [20]:
b

[290, 430, 360]

##Реализация

In [21]:
def get_simplex_table(f, A, b):
  xb = [eq + [x] for eq, x in zip(A, b)]
  z = f + [0]
  return xb + [z]

In [22]:
def can_be_better(simplex_table):
    z = simplex_table[-1]
    return any(x > 0 for x in z[:-1])

In [23]:
def get_pivot_position(simplex_table):
    z = simplex_table[-1]
    column = next(i for i, x in enumerate(z[:-1]) if x > 0)

    restrictions = []
    for eq in simplex_table[:-1]:
        el = eq[column]
        restrictions.append(math.inf if el <= 0 else eq[-1] / el)

    row = restrictions.index(min(restrictions))
    return row, column

In [24]:
def simplex_step(simplex_table, pivot_position):
    new_simplex_table = [[] for eq in simplex_table]

    i, j = pivot_position
    pivot_value = simplex_table[i][j]
    new_simplex_table[i] = np.array(simplex_table[i]) / pivot_value

    for eq_i, eq in enumerate(simplex_table):
        if eq_i != i:
            multiplier = np.array(new_simplex_table[i]) * simplex_table[eq_i][j]
            new_simplex_table[eq_i] = np.array(simplex_table[eq_i]) - multiplier

    return new_simplex_table

In [25]:
def is_basic(column):
    return sum(column) == 1 and len([c for c in column if c == 0]) == len(column) - 1

def get_solution(simplex_table):
    columns = np.array(simplex_table).T
    solutions = []
    for column in columns:
        solution = 0
        if is_basic(column):
            one_index = column.tolist().index(1)
            solution = columns[-1][one_index]
        solutions.append(solution)

    return solutions

In [26]:
def simplex_method(f, A, b):
    simplex_table = get_simplex_table(f, A, b)

    while can_be_better(simplex_table):
        pivot_position = get_pivot_position(simplex_table)
        simplex_table = simplex_step(simplex_table, pivot_position)

    return get_solution(simplex_table)

In [29]:
solution = simplex_method(f, A, b)
print('solution: ', solution[:2])

solution:  [79.47368421052632, 42.10526315789474]


In [30]:
print(f'objective function value: {f[0]*(solution[0]) + f[1]*(solution[1]):.2f}')

objective function value: 285.26
